In [1]:
import pandas as pd
import geopandas as gp
import folium
import numpy as np 
from sklearn.datasets import load_iris
import seaborn as sns

%matplotlib inline

In [2]:
com_area = gp.read_file('C:\Workspace\TNC-Demand-Model\Inputs\Chicago Community Areas\geo_export_d8da94d2-4ef1-4ee6-9fff-7bb20d451fe2.shp')


In [3]:
com_area.area_num_1 = com_area.area_num_1.astype(float)


In [4]:
com_area_conflate = pd.read_csv('C:\Workspace\TNC-Demand-Model\Inputs\Chicago Community Areas\Community_Area_to_Census_Tract.csv')


In [5]:
tods = [1,2,3,4,5]
df3 = pd.DataFrame()
df4 =  pd.DataFrame()
df5 = pd.DataFrame()

In [7]:
for tod in tods:
    print('Working on TOD ' + str(tod))
    df = pd.read_hdf('C:\Workspace\TNC-Demand-Model\Outputs\Chicago_TNC_Trips_20.H5', columns = ['Pickup Census Tract','Dropoff Census Tract','MONTH','YEAR','Pickup Community Area','Dropoff Community Area','Trip Seconds','Trip Miles','Fare'],key = 'Weekday_' + str(tod))
    df2 = df[(~np.isnan(df['DEST_COM_AREA']))&(~np.isnan(df['ORIGIN_COM_AREA']))]
    df2 = df2[(np.isnan(df2['DESTINATION']))|(np.isnan(df2['ORIGIN']))]
    df2['SUPPRESSED_TRIPS'] = 1
   # df2['DAY'] = df2['Trip Start Timestamp'].dt.day
    #df2['MINUTE'] = df2['Trip Start Timestamp'].dt.minute


    df_unsup = df[(~np.isnan(df['DEST_COM_AREA']))&(~np.isnan(df['ORIGIN_COM_AREA']))]
    df_unsup = df_unsup[(~np.isnan(df_unsup['DESTINATION']))&(~np.isnan(df_unsup['ORIGIN']))]
    df_unsup['UNSUPPRESSED_TRIPS'] = 1
    #df_unsup['DAY'] = df_unsup['Trip Start Timestamp'].dt.day
    #df_unsup['MINUTE'] = df_unsup['Trip Start Timestamp'].dt.minute

    df_external = df[(np.isnan(df['ORIGIN_COM_AREA']))|(np.isnan(df['DEST_COM_AREA']))]
    df_external['EXTERNAL_TRIPS'] = 1

    df3 = df3.append(df2)
    df4 = df4.append(df_unsup)
    df5 = df5.append(df_external)

#sup_pickup_daily = df3.groupby(by = ['MONTH','YEAR','DAY','Pickup Community Area'],as_index = False).sum()
sup_pickup_monthly_sum = df3.groupby(by = ['MONTH','YEAR','ORIGIN_COM_AREA'],as_index = False).sum()
#sup_pickup_monthly_avg = sup_pickup_daily.groupby(by = ['MONTH','YEAR','Pickup Community Area'],as_index = False).mean()
#sup_dropoff_daily = df3.groupby(by = ['MONTH','YEAR','DAY','Dropoff Community Area'],as_index = False).sum()
#sup_dropoff_monthly_sum = df3.groupby(by = ['MONTH','YEAR','Dropoff Community Area'],as_index = False).sum()
#sup_dropoff_monthly_avg = sup_dropoff_daily.groupby(by = ['MONTH','YEAR','Dropoff Community Area'],as_index = False).mean()

Working on TOD 1


KeyError: 'DEST_COM_AREA'

In [ ]:
unsup_pickup_monthly_sum = df4.groupby(by = ['MONTH','YEAR','ORIGIN_COM_AREA'],as_index = False).sum()
df_external_pickup_sum = df5.groupby(by = ['MONTH','YEAR','ORIGIN_COM_AREA'],as_index = False).sum()


join_pickup = sup_pickup_monthly_sum.merge(unsup_pickup_monthly_sum, on = ['MONTH','YEAR','ORIGIN_COM_AREA'])
join_pickup = join_pickup.merge(df_external_pickup_sum, on = ['MONTH','YEAR','ORIGIN_COM_AREA'])